# EDA Final Data

- 2017, 2018년도 6,7,8월 데이터

## 1. Load Data & Default Pre-processing

## 1) Load Data

In [1]:
import pandas as pd
import sagemaker
import numpy as np

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### - Data Load From S3

In [3]:
# # # S3 bucket
# bucket = 'poc-2209-twayairport-dp'

# # 예약데이터
# booking_df = pd.read_csv(f's3://{bucket}/data/original/예약_20170608.csv')
# booking_df = pd.concat([booking_df, pd.read_csv(f's3://{bucket}/data/original/예약_20180608.csv')])
# booking_df = pd.concat([booking_df, pd.read_csv(f's3://{bucket}/data/original/예약_20220220_20220226.csv')])
# booking_df = pd.concat([booking_df, pd.read_csv(f's3://{bucket}/data/original/예약_20220821_20220827.csv')])

# # 쿠폰데이터
# coupon_df = pd.read_csv(f's3://{bucket}/data/original/쿠폰_20170608_20180608.csv')
# coupon_df = pd.concat([coupon_df, pd.read_csv(f's3://{bucket}/data/original/쿠폰_20220220_20220226.csv')])
# coupon_df = pd.concat([coupon_df, pd.read_csv(f's3://{bucket}/data/original/쿠폰_20220821_20220827.csv')])

# # 비행편데이터
# flight_df = pd.read_csv(f's3://{bucket}/data/original/비행편정보_20170608_20180608.csv')
# flight_df = pd.concat([flight_df, pd.read_csv(f's3://{bucket}/data/original/비행편정보_202202_202208.csv')])

# # 기타 정보
# discount_ratio = pd.read_csv(f's3://{bucket}/data/original/항공권할인기준.csv')
# peak_low_season = pd.read_csv(f's3://{bucket}/data/original/all_peak_low_season.csv')
# fare_by_class = pd.read_csv(f's3://{bucket}/data/original/3차_fare_list.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### - Data Load as parquet

In [35]:
# data_list = ['booking_df', 'coupon_df', 'flight_df', 'discount_ratio', 'peak_low_season', 'fare_by_class', 'total_df']
# for i in data_list:
#     print(f"Data {i} Read Done -----------")
#     globals()[i] = pd.read_parquet(f'/root/data/{i}.parquet')

Data booking_df Read Done -----------
Data coupon_df Read Done -----------


## 2) 기본 전처리 작업

In [5]:
# 컬럼명 변경은 핵심적으로 사용할 _ 표현식을 기준으로함.
coupon_df.rename(columns={
                    'flightno': 'flight_number',
                    'flightdate': 'flight_departure_date',
                    'issuedate': 'capture_date',
                    'carrier': 'airline_code',
                    'bookingclass': 'fare_class',
                    'cabinclass': 'cabin_code',
                  }, inplace=True)

flight_df.rename(columns={'cabin_class': 'cabin_code'}, inplace=True)

In [6]:
# 날짜 데이터 형변환
booking_df['flight_departure_date'] = pd.to_datetime(booking_df['flight_departure_date'], format='%Y-%m-%d')
booking_df['capture_date'] = pd.to_datetime(booking_df['capture_date'], format='%Y-%m-%d')

coupon_df['flight_departure_date'] = coupon_df['flight_departure_date'].astype('str').str.replace('-','')
coupon_df['flight_departure_date'] = pd.to_datetime(coupon_df['flight_departure_date'], format='%Y-%m-%d')
coupon_df['capture_date'] = coupon_df['capture_date'].astype('str').str.replace('-','')
coupon_df['capture_date'] = pd.to_datetime(coupon_df['capture_date'], format='%Y-%m-%d')

flight_df['flight_departure_date'] = pd.to_datetime(flight_df['flight_departure_date'], format='%Y-%m-%d')
flight_df['leg_departure_date_time'] = pd.to_datetime(flight_df['leg_departure_date_time'])
flight_df['leg_arrival_date_time'] = pd.to_datetime(flight_df['leg_arrival_date_time'])

peak_low_season['peak_season_s'] = pd.to_datetime(peak_low_season['peak_season_s'],  format='%Y-%m-%d')
peak_low_season['peak_season_e'] = pd.to_datetime(peak_low_season['peak_season_e'],  format='%Y-%m-%d')
peak_low_season['low_season_s'] = pd.to_datetime(peak_low_season['low_season_s'],  format='%Y-%m-%d')
peak_low_season['low_season_e'] = pd.to_datetime(peak_low_season['low_season_e'],  format='%Y-%m-%d')

## 3) 데이터 필터링, 파생변수 생성, 가격 데이터 조정

### - 데이터 필터링(이코노미 데이터)

In [7]:
# bookingclass 특수값
business = ['F', 'C', 'R', 'CC', 'CG', 'C1', 'C2'] # 이코너미만 체크할 것
economic = ['Y', 'Z', 'W', 'V', 'T', 'S', 'P', 'O', 'N', 'M', 'L', 'K', 'I', 'H', 'J', 'E', 'D', 'B']
event = ['A', 'BB', 'BL', 'UU', 'U',  'G', 'Q', 'R', 'X'] # 정상가가 아님 고려대상 X

In [8]:
# bookingclass와 cabincass 모두 이코노미 데이터만 확인
coupon_df = coupon_df[(coupon_df['fare_class'].isin(economic))&(coupon_df['cabin_code'] == 'Y')].reset_index(drop = True)
booking_df = booking_df[(booking_df['cabin_code'] == 'Y')].reset_index(drop = True) ## 잔여좌석 계산때문에 economic fare_class는 나중에 제외
flight_df = flight_df[flight_df['cabin_code'] == 'Y'].reset_index(drop = True)

### - 파생변수 생성 (year)

In [9]:
coupon_df['year'] = coupon_df['flight_departure_date'].dt.year ## 항공권 출발 연도

### - 가격 데이터 조정

In [10]:
## 성수기 일자 생성
peak_range = []

for i in range(len(peak_low_season)):
    peak_range.extend(pd.date_range(peak_low_season['peak_season_s'].iloc[i], peak_low_season['peak_season_e'].iloc[i]))

In [11]:
## 할인율 데이터 조인
coupon_df = pd.merge(coupon_df, discount_ratio, on=['year', 'discountcode'], how = 'left')

In [12]:
coupon_df['peak_season'] = coupon_df['peak_season'].fillna(0)
coupon_df['low_season'] = coupon_df['low_season'].fillna(0)
## Discount code 별 할인정보 반영
coupon_df['fix_fare'] = np.where(coupon_df['flight_departure_date'].isin(peak_range),
                                 np.where((coupon_df['peak_season'] != 0) & (coupon_df['fare_class'] == 'Y'), coupon_df['fare']/(1-coupon_df['peak_season']), coupon_df['fare']),
                                 np.where((coupon_df['low_season'] != 0) & (coupon_df['fare_class'] == 'Y'), coupon_df['fare']/(1-coupon_df['low_season']), coupon_df['fare']))
## 어린이 가격 할인정보 반영
coupon_df['fix_fare'] = np.where((~coupon_df['fare_class'].isin(['A','B','D','E'])) & (coupon_df['discountcode'] == 'CH'),
                                 coupon_df['fix_fare']+5000, coupon_df['fix_fare'])
coupon_df = coupon_df[coupon_df['fix_fare'] != 0]
coupon_df = coupon_df[coupon_df['fix_fare'].notna()]

### - 전체 데이터 조인

In [13]:
orig_df = pd.merge(booking_df, coupon_df, on=list(set(list(booking_df)) & set(list(coupon_df))), how = 'right')
orig_df.dropna(axis = 0, inplace = True)
orig_df = pd.merge(flight_df, orig_df, on = list(set(list(flight_df)) & set(list(orig_df))), how = 'right')
orig_df.reset_index(drop = True, inplace = True)    
print(f"Coupon & Booking & Flight Data merged.")
print(orig_df.shape)

Coupon & Booking & Flight Data merged.
(704981, 33)


### - 파생변수 생성 (누적예약수, 잔여 좌석, 잔여 일자 등등..)

In [14]:
# 누적예약 수
seats_df = pd.DataFrame({'total_sold_seats' : booking_df.groupby(["flight_number", "capture_date", 'cabin_code', 'flight_departure_date'])['sold_seats'].sum()}).reset_index()

In [15]:
df = pd.DataFrame()
for i in seats_df['flight_number'].unique():
    for j in seats_df['flight_departure_date'].unique():
        temp_df = seats_df[(seats_df['flight_number'] == i)&(seats_df['flight_departure_date'] == j)]
        temp_df.sort_values('capture_date', inplace = True)
        temp_df.reset_index(drop = True, inplace = True)
        temp_df['before_tss'] = temp_df['total_sold_seats'] ## 전일 누적 예약 수                
        temp_df['before_ss'] = temp_df['before_tss'] - temp_df['total_sold_seats'].shift(2) ## 전일 판매건수
        temp_df.fillna(0, inplace = True)
        df = pd.concat([df, temp_df])        

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [16]:
total_df = pd.merge(df, orig_df, on = list(set(list(df)) & set(list(orig_df))), how = 'right')

In [17]:
total_df = pd.merge(total_df, pd.get_dummies(total_df['startseg']), right_index = True, left_index = True)
total_df['remain_seats'] = total_df['cabin_authorised_cap'] - total_df['before_tss'] ## 전일 잔여 좌석 수
total_df['departure_hour'] = total_df['leg_departure_date_time'].dt.hour ## 출발 시간
total_df['departure_minute'] = total_df['leg_departure_date_time'].dt.minute ## 출발 분
total_df['departure_time_format_min'] = total_df['departure_hour'] * 60 + total_df['departure_minute'] ## 출발 시간+분 -> 분
total_df['issueweekday'] = total_df['capture_date'].dt.weekday ## 티켓 구매 요일
total_df['departureweekday'] = total_df['leg_departure_date_time'].dt.weekday ## 항공권 출발 요일
total_df['remain_days'] = total_df['flight_departure_date'] - total_df['capture_date'] ## 출발까지 잔여일
total_df['remain_days'] = total_df['remain_days'].astype('str').str.split().str[0]
total_df['remain_days'] = total_df['remain_days'].astype('int')
total_df['month'] = total_df['flight_departure_date'].dt.month
total_df['peak'] = np.where(total_df['flight_departure_date'].isin(peak_range), 1, 0)

### - 파생변수 생성(주중/주말/주말할증)

- 2017 : 주중(월, 화, 수, 목) // 주말(금, 토, 일) // 주말할증(김포출발 : 금, 토 12시이전 & 제주출발 : 일 12시 이후)
- 2018 : 주중(월, 화, 수, 목) // 주말(금, 토, 일) // 주말할증(김포출발 : 금, 토 & 제주출발 : 일)
- 2022 : 
    김포출발 - 주중(월, 화, 수, 목) // 주말(일) // 주말할증(금, 토)
    제주출발 - 주중(화, 수, 목) // 주말(월, 금, 토) // 주말할증(일)

In [18]:
premium_list = pd.read_csv('s3://poc-2209-twayairport-dp/data/original/premium_list.csv', encoding='utf-8')
total_df = pd.merge(total_df, premium_list, on = ['year', 'startseg', 'departureweekday'])

In [19]:
total_df.reset_index(drop = True, inplace = True)

In [20]:
for i in total_df[total_df['option'].notna()].index:
    if (total_df.loc[i,'startseg'] == 'GMP') & (total_df.loc[i,'departure_time_format_min'] >= 720):
        total_df.loc[i,'premium'] = 0
    elif (total_df.loc[i,'startseg'] == 'CJU') & (total_df.loc[i,'departure_time_format_min'] < 720):
        total_df.loc[i,'premium'] = 0
    else:
        pass        

In [21]:
## 단일값 제거
print([i for i in list(total_df) if total_df[i].nunique() == 1 ])
total_df.drop([i for i in list(total_df) if total_df[i].nunique() == 1 ], axis = 1, inplace = True)

['cabin_code', 'airline_code', 'operating_airline_code', 'fare_class_line']


### - Dummy 변수 생성

In [22]:
dummy_list = ['departureweekday', 'issueweekday', 'month', 'year']
for i in dummy_list:
    dummy_data = pd.get_dummies(total_df[i])        
    dummy_data = pd.DataFrame(columns = [f'{i}_{j}' for j in list(dummy_data)], data = dummy_data.values)
    dummy_data.reset_index(drop = True, inplace = True)
    total_df = pd.merge(total_df, dummy_data, right_index = True, left_index = True)
    total_df.reset_index(drop = True, inplace = True)

### - 2차 전처리 데이터 저장

In [23]:
data_list = ['booking_df', 'coupon_df', 'flight_df', 'discount_ratio', 'peak_low_season', 'fare_by_class', 'total_df']
for i in data_list:
    globals()[i].to_parquet(f'/root/data/{i}.parquet')
    globals()[i].to_parquet(f's3://poc-2209-twayairport-dp/data/original/{i}.parquet')

## 5) EDA

### - 상관관계 분석

In [28]:
# cor_matrix = total_df[features].corr().abs()
cor_matrix = total_df.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
high_score = [column for column in upper_tri.columns if any(upper_tri[column] > 0.80)]    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
high_score

['before_tss',
 'authorised_cap',
 'farevat',
 'fscvat',
 'low_season',
 'fix_value',
 'fix_fare',
 'GMP',
 'remain_seats',
 'departure_time_format_min',
 'weekday',
 'weekend',
 'year_2017',
 'year_2018',
 'year_2022']

In [30]:
cor_matrix.loc[high_score,high_score].style.background_gradient(cmap='coolwarm')

,before_tss,authorised_cap,farevat,fscvat,low_season,fix_value,fix_fare,GMP,remain_seats,departure_time_format_min,weekday,weekend,year_2017,year_2018,year_2022
before_tss,1.000000,0.821337,0.151269,0.137402,0.046447,0.104462,0.159112,0.006709,0.943489,0.031137,0.025086,0.005466,0.091707,0.114348,0.046140
authorised_cap,0.821337,1.000000,0.109664,0.183913,0.034462,0.088916,0.116444,0.015455,0.771438,0.017927,0.008135,0.011217,0.040084,0.036293,0.004128
farevat,0.151269,0.109664,1.000000,0.171125,0.044305,0.034810,0.997281,0.009104,0.119760,0.025366,0.349864,0.278368,0.134563,0.060678,0.122420
fscvat,0.137402,0.183913,0.171125,1.000000,0.022921,0.061833,0.172591,0.021023,0.015125,0.003868,0.053536,0.038793,0.543217,0.155844,0.652366
low_season,0.046447,0.034462,0.044305,0.022921,1.000000,0.071720,0.049456,0.009370,0.049022,0.006962,0.003853,0.011657,0.021597,0.000659,0.035765
fix_value,0.104462,0.088916,0.034810,0.061833,0.071720,1.000000,0.033200,0.005059,0.093512,0.003852,0.000279,0.001861,0.023569,0.077271,0.096855
fix_fare,0.159112,0.116444,0.997281,0.172591,0.049456,0.033200,1.000000,0.010009,0.127339,0.025281,0.349950,0.278458,0.132204,0.057617,0.123818
GMP,0.006709,0.015455,0.009104,0.021023,0.009370,0.005059,0.010009,1.000000,0.003925,0.032810,0.061145,0.045975,0.034216,0.035240,0.004038
remain_seats,0.943489,0.771438,0.119760,0.015125,0.049022,0.093512,0.127339,0.003925,1.000000,0.030543,0.025016,0.002302,0.124201,0.084345,0.062687
departure_time_format_min,0.031137,0.017927,0.025366,0.003868,0.006962,0.003852,0.025281,0.032810,0.030543,1.000000,0.001920,0.008614,0.000233,0.010799,0.019566


### - 이상치 파악

In [31]:
## 신규 항공권 확인
temp_list = [[i, total_df[total_df['flight_number'] == i]['remain_days'].max()] for i in total_df['flight_number'].unique()]
temp = pd.DataFrame(columns = ['flight_number', 'remain_max'], data = temp_list)
temp.sort_values('remain_max',inplace = True)
rm_flight = temp[:10]['flight_number'].tolist()
temp

,flight_number,remain_max
49,9748,28
33,9732,29
52,9705,34
57,9706,35
48,9741,35
43,742,82
38,746,93
41,748,97
42,772,99
39,752,101


In [32]:
## Boxplot 기준 이상치 확인
for month in [2,6,7,8]:
    for gimpo_jeju in [0,1]: ## 도착지 순서(0이면 김포 도착, 1이면 제주 도착)
        try:
            temp = total_df[(total_df['month'] == month) & (total_df['flight_number'] % 2 == gimpo_jeju)] ## 2월 제주행
            arr1 = temp['fix_fare']

            q1 = np.quantile(arr1, 0.25)
            q3 = np.quantile(arr1, 0.75)
            med = np.median(arr1)
            iqr = q3-q1

            upper_bound = q3+(1.5*iqr)
            lower_bound = q1-(1.5*iqr)
            outliers = arr1[(arr1 <= lower_bound) | (arr1 >= upper_bound)]

            if gimpo_jeju == 0:
                region = 'GMP'
            else:
                region = 'CJU'
            print(f'{month} month dest {region} : {outliers.unique()} // {round(temp[temp["fix_fare"].isin(outliers.unique())].shape[0]/temp.shape[0]*100, 2)} %')
            # plt.boxplot(temp['fix_fare'])
            # plt.show()
            print("*"*100)
        except:
            pass

2 month dest GMP : [ 95900.  89900. 101900.  87900.  87210.  87360.] // 2.73 %
****************************************************************************************************
2 month dest CJU : [ 95900. 101900.] // 2.42 %
****************************************************************************************************
6 month dest GMP : [] // 0.0 %
****************************************************************************************************
6 month dest CJU : [] // 0.0 %
****************************************************************************************************
7 month dest GMP : [] // 0.0 %
****************************************************************************************************
7 month dest CJU : [] // 0.0 %
****************************************************************************************************
8 month dest GMP : [] // 0.0 %
****************************************************************************************************
8 month dest

### - 일자별 누적 판매 건수 평균

In [33]:
plt.figure(figsize = (15,5))
for month in [6]:
    for year in [2017,2018]:
        for startseg in ['GMP', 'CJU']:
            temp = total_df[(total_df['month'] == month) & (total_df['year'] == year) & (total_df['startseg'] == startseg)].copy()
            # temp = temp[~temp['flight_number'].isin(rm_flight)].copy()
            temp = pd.DataFrame(temp.groupby(['capture_date'])['total_sold_seats'].mean())        
            plt.plot(temp.index, temp['total_sold_seats'], label = f"{month} month {region} startseg")
plt.ylim(0,200)        
plt.legend()
plt.show()

NameError: name 'plt' is not defined

### - 일자별 구매 금액 평균

In [ ]:
plt.figure(figsize = (15,5))
for month in [6]:
    for year in [2017,2018]:
        for startseg in ['GMP', 'CJU']:
            temp = total_df[(total_df['month'] == month) & (total_df['year'] == year) & (total_df['startseg'] == startseg)].copy()
            # temp = temp[~temp['flight_number'].isin(rm_flight)].copy()
            temp = pd.DataFrame(temp.groupby(['capture_date'])['fix_fare'].mean())        
            plt.plot(temp.index, temp['fix_fare'], label = f"{month} month {startseg} start")
# plt.ylim(0,200)        
plt.legend()
plt.show()